In [ ]:
import xarray as xr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, sys, yaml
USER = os.getenv('USER')
from dscim.menu.simple_storage import Climate
from itertools import product

In [ ]:
sectors=['CAMEL_clipped']
kinds=['zeroes']#, 'p0.01', 'p99.99']
masks=['epa_mask'] #, 'gdppc_mask', 'unmasked']
eta_rhos = {
    # 2.0 :0.0,
    1.016010255: 9.149608e-05,
    # 1.244459066: 0.00197263997,
    # 1.421158116: 0.00461878399,
    # 1.567899395: 0.00770271076,
}

In [ ]:
def timeseries(
    sector,
    mask,
    kind,
    recipe='risk_aversion',
    disc='euler_ramsey',
    eta=2.0,
    rho=0.0,
):
    
    config = f'/home/{USER}/repos/integration/configs/rff_config.yaml'
    results = f"/mnt/CIL_integration/rff3_with_disc_factors/iter0-19/{sector}/2020/unmasked_None"
    
    rff_ids = pd.read_csv(
        f"{results}/rff_ids/{recipe}_{disc}_eta{eta}_rho{rho}_{kind}_{mask}_rff_ids.csv"
    ).set_index(['rff_sp', 'simulation']).index

    sccs = xr.open_dataset(
        f"{results}/{recipe}_{disc}_eta{eta}_rho{rho}_uncollapsed_sccs.nc4"
    ).sel(weitzman_parameter='0.5', fair_aggregation='uncollapsed', drop=True).uncollapsed_sccs.stack(
        {'rff_sp-simulation' :['rff_sp', 'simulation']}
    ).sel({'rff_sp-simulation' : rff_ids}
         ).rename('SCC').to_dataframe().reset_index()

    fig, ax = plt.subplots(3,1,figsize=(15,15), sharex=True)
    
    for i, yvar in enumerate(['cumulative_emissions', 'emissions', 'gmst']):
        if yvar=='cumulative_emissions':
            emissions = xr.open_dataset("/shares/gcp/integration/rff/climate/rff-sp_emissions_all_gases.nc")
            c_emissions=emissions.cumsum('Year')
            c_emissions['Year'] = emissions.Year
            ydata = (c_emissions
                        .sel(gas='C', simulation=rff_ids.get_level_values('rff_sp'), drop=True)
                        .to_dataframe()
                        .reset_index()
                        .rename(columns={'simulation' : 'rff_sp', 'Year' : 'year', 'emissions' : 'cumulative_emissions'})
                       )
            idx=['rff_sp']
        elif yvar=='emissions':
            ydata = (xr.open_dataset(
                        "/shares/gcp/integration/rff/climate/rff-sp_emissions_all_gases.nc")
                        .sel(gas='C', simulation=rff_ids.get_level_values('rff_sp'), drop=True)
                        .to_dataframe()
                        .reset_index()
                        .rename(columns={'simulation' : 'rff_sp', 'Year' : 'year'})
                       )
            idx=['rff_sp']
        elif yvar=='gmst':
            with open(config) as config_file:
                params = yaml.full_load(config_file)

            ydata = (Climate(**params['climate'])
                     .fair_pulse
                     .stack({'rff_sp-simulation' :['rff_sp', 'simulation']})
                     .sel({'rff_sp-simulation' : rff_ids})
                     .to_dataframe()
                     .reset_index()
                     .rename(columns={'temperature' : 'gmst'})
                    )
            idx=['rff_sp', 'simulation']

        data = sccs.merge(ydata, on=idx)

        sns.lineplot(data=data,
                 x='year',
                 y=yvar,
                 hue='SCC',
                 ax=ax[i]
                )
        
        ax[i].set_title(yvar)
        if i > 0:
            ax[i].get_legend().remove()
    
    plt.subplots_adjust(top=0.85)
    fig.suptitle(f"mask={mask}, SCC subset={kind} \n {sector} {recipe} {disc}, eta={eta} rho={rho}")
    plt.savefig(
        f'/mnt/CIL_integration/rff_diagnostics/v3/climate_timeseries/climate_timeseries_{sector}_{recipe}_{disc}_{eta}_{rho}_{kind}_{mask}.png', 
        bbox_inches='tight', 
        dpi=300
        )

In [ ]:
for s, m, k, er in product(sectors, masks, kinds, eta_rhos.items()):
    print(s,m,k,er)
    timeseries(sector=s,
               mask=m,
                    kind=k,
                    eta=er[0],
                    rho=er[1],
                   )